In [ ]:
# === CONFIGURATION ===

SYMBOLS = ['DOGE/USDT', 'BTC/USDT', 'ETH/USDT', 'TIA/USDT', 'SOL/USDT', 'XRP/USDT', 'ADA/USDT']
TIMEFRAMES = ['15m', '1h']
EXCHANGE = 'bybit'
TAKE_PROFIT_PCT = 3       # Take profit at +3%
STOP_LOSS_PCT = 1.5       # Stop loss at -1.5%
SLEEP_TIME = 60           # Seconds between scans

# === IMPORTS ===
import ccxt
import time
import pandas as pd
import requests
from ta.momentum import RSIIndicator, StochRSIIndicator
from ta.trend import ADXIndicator
from ta.volume import AccDistIndexIndicator

# === INIT EXCHANGE ===
exchange = getattr(ccxt, EXCHANGE)()
exchange.load_markets()

# Track open trades
open_trades = {}  # {symbol: {"side": "long/short", "entry": price, "tp": tp, "sl": sl}}

# === FETCH OHLCV ===
def fetch_ohlcv(symbol, timeframe='15m', limit=100):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# === CALCULATE INDICATORS ===
def calculate_indicators(df):
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    stoch = StochRSIIndicator(df['close'], window=14)
    df['stoch_rsi_k'] = stoch.stochrsi_k()
    df['stoch_rsi_d'] = stoch.stochrsi_d()
    df['adx'] = ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    df['ad'] = AccDistIndexIndicator(df['high'], df['low'], df['close'], df['volume']).acc_dist_index()
    return df

# === SIGNAL LOGIC (Improved with Crossover) ===
def generate_signal(df_15m, df_1h):
    latest = df_15m.iloc[-1]
    h1 = df_1h.iloc[-1]

    # Trend filter with EMA (higher timeframe)
    df_1h['ema50'] = df_1h['close'].ewm(span=50).mean()
    h1_ema = df_1h['ema50'].iloc[-1]

    trend_up = h1['close'] > h1_ema
    trend_down = h1['close'] < h1_ema

    # RSI + Stoch RSI crossover check
    rsi_prev_15m = df_15m['rsi'].iloc[-2]
    rsi_now_15m = latest['rsi']
    stoch_prev_15m = df_15m['stoch_rsi_k'].iloc[-2]
    stoch_now_15m = latest['stoch_rsi_k']

    # Long entry
    long_signal = (
        trend_up and
        rsi_prev_15m < 50 and rsi_now_15m > 50 and
        stoch_prev_15m < 0.5 and stoch_now_15m > 0.5 and
        h1['adx'] > 20
    )

    # Short entry
    short_signal = (
        trend_down and
        rsi_prev_15m > 50 and rsi_now_15m < 50 and
        stoch_prev_15m > 0.5 and stoch_now_15m < 0.5 and
        h1['adx'] > 20
    )

    # Exit logic (opposite cross)
    exit_long = (
        rsi_prev_15m > 50 and rsi_now_15m < 50 and
        stoch_prev_15m > 0.5 and stoch_now_15m < 0.5
    )

    exit_short = (
        rsi_prev_15m < 50 and rsi_now_15m > 50 and
        stoch_prev_15m < 0.5 and stoch_now_15m > 0.5
    )

    print("\n--- Indicator Debug ---")
    print(f"15m RSI: {rsi_now_15m:.2f} (prev {rsi_prev_15m:.2f}), "
          f"Stoch RSI K: {stoch_now_15m:.2f} (prev {stoch_prev_15m:.2f}), "
          f"ADX: {latest['adx']:.2f}")
    print(f"1h  RSI: {h1['rsi']:.2f}, ADX: {h1['adx']:.2f}, EMA50: {h1_ema:.2f}, Close: {h1['close']:.2f}")
    print("-----------------------")

    if long_signal:
        return "long"
    elif short_signal:
        return "short"
    elif exit_long:
        return "exit_long"
    elif exit_short:
        return "exit_short"
    return None

# === TELEGRAM ALERT ===
def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    data = {'chat_id': CHAT_ID, 'text': message}
    try:
        response = requests.post(url, data=data)
        if response.status_code != 200:
            print("Telegram Error:", response.text)
    except Exception as e:
        print("Telegram Exception:", e)

# === TRADE EXECUTION ===
def execute_trade(symbol, signal):
    ticker = exchange.fetch_ticker(symbol)
    price = ticker['last']

    if signal == 'long':
        sl = price * (1 - STOP_LOSS_PCT/100)
        tp = price * (1 + TAKE_PROFIT_PCT/100)
        open_trades[symbol] = {"side": "long", "entry": price, "tp": tp, "sl": sl}
        msg = f"✅ LONG ENTRY: {symbol} at {price:.4f}\n🎯 TP: {tp:.4f}, 🛑 SL: {sl:.4f}"

    elif signal == 'short':
        sl = price * (1 + STOP_LOSS_PCT/100)
        tp = price * (1 - TAKE_PROFIT_PCT/100)
        open_trades[symbol] = {"side": "short", "entry": price, "tp": tp, "sl": sl}
        msg = f"✅ SHORT ENTRY: {symbol} at {price:.4f}\n🎯 TP: {tp:.4f}, 🛑 SL: {sl:.4f}"

    elif signal == 'exit_long' and symbol in open_trades and open_trades[symbol]["side"] == "long":
        msg = f"🚪 EXIT LONG: {symbol} at {price:.4f} (RSI/Stoch flipped)"
        del open_trades[symbol]

    elif signal == 'exit_short' and symbol in open_trades and open_trades[symbol]["side"] == "short":
        msg = f"🚪 EXIT SHORT: {symbol} at {price:.4f} (RSI/Stoch flipped)"
        del open_trades[symbol]

    else:
        return  # No action

    send_telegram_message(msg)
    print(msg)

# === MAIN LOOP ===
def run_bot():
    while True:
        for symbol in SYMBOLS:
            try:
                print(f"\n🔄 Fetching data for {symbol}")
                df_15m = calculate_indicators(fetch_ohlcv(symbol, '15m'))
                df_1h = calculate_indicators(fetch_ohlcv(symbol, '1h'))

                signal = generate_signal(df_15m, df_1h)
                print(f"📊 adjusted Signal for {symbol}: {signal}")

                if signal:
                    execute_trade(symbol, signal)
                else:
                    print(f"⛔ No actionable signal for {symbol}")

            except Exception as e:
                print(f"❌ Error for {symbol}: {e}")

        print(f"⏳ Waiting {SLEEP_TIME} seconds...\n")
        time.sleep(SLEEP_TIME)

# === RUN ===
if __name__ == '__main__':
    run_bot()


In [ ]:
import requests
import pandas as pd
import ta
import ccxt
import time
import os
from datetime import datetime
from playsound import playsound
from plyer import notification

# === CONFIG ===
symbols = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT', 'WIF/USDT', 'TIA/USDT', 'XRP/USDT']
exchange = ccxt.bybit({'enableRateLimit': True})

alert_url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
sound_file = "alert.mp3"
VOLUME_SPIKE_THRESHOLD = 1.05
ALERT_COOLDOWN = 300  # 5 minutes
last_alert_time = {}

# === ALERT FUNCTION ===
def send_alert(symbol, msg, tv_symbol):
    time_now = datetime.now().strftime('%H:%M:%S')
    chart_link = f"https://www.tradingview.com/chart/?symbol=BYBIT:{tv_symbol}"
    full_msg = f"[{time_now}] - {symbol}\n{msg}\n📊 Chart: {chart_link}"

    try:
        requests.post(alert_url, data={"chat_id": chat_id, "text": full_msg})
        print(f"📢 Sent: {full_msg}")
        
        with open("alerts_log.txt", "a") as f:
            f.write(full_msg + "\n")

        if os.path.exists(sound_file):
            playsound(sound_file)
        else:
            print("🔇 alert.mp3 not found.")

        notification.notify(title=f"{symbol} SIGNAL", message=msg, timeout=5)

    except Exception as e:
        print(f"[Alert error] {e}")

# === OHLC FETCH ===
def fetch_ohlcv(symbol, tf='15m', limit=100):
    try:
        data = exchange.fetch_ohlcv(symbol, timeframe=tf, limit=limit)
        df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
        return df
    except:
        return None

# === CANDLESTICK DETECTOR ===
def detect_candlestick(df):
    patterns = []
    for i in range(len(df)):
        if i == 0:
            patterns.append("")
            continue
        candle = ''
        body = abs(df['close'][i] - df['open'][i])
        range_ = df['high'][i] - df['low'][i]
        if range_ == 0:
            patterns.append("")
            continue
        if body < 0.2 * range_:
            candle = 'Doji'
        elif df['close'][i] > df['open'][i] and df['open'][i-1] > df['close'][i-1] and df['close'][i] > df['open'][i-1]:
            candle = 'Bullish Engulfing'
        elif df['close'][i] < df['open'][i] and df['open'][i-1] < df['close'][i-1] and df['close'][i] < df['open'][i-1]:
            candle = 'Bearish Engulfing'
        patterns.append(candle)
    return patterns

# === INDICATOR ANALYSIS ===
def analyze(df):
    df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'], window=14, smooth_window=3)
    df['stoch_k'] = stoch.stoch()
    df['stoch_d'] = stoch.stoch_signal()
    df['adx'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    df['ema_fast'] = ta.trend.EMAIndicator(df['close'], window=9).ema_indicator()
    df['ema_slow'] = ta.trend.EMAIndicator(df['close'], window=21).ema_indicator()
    df['vol_ma'] = df['volume'].rolling(20).mean()
    df['pattern'] = detect_candlestick(df)
    return df.dropna().reset_index(drop=True)

# === SIGNAL CHECK ===
def check_signals(symbol):
    global last_alert_time
    df_15m = fetch_ohlcv(symbol, '15m')
    df_1h = fetch_ohlcv(symbol, '1h')

    if df_15m is None or df_1h is None:
        return

    df_15m = analyze(df_15m)
    df_1h = analyze(df_1h)

    try:
        latest = df_15m.iloc[-1]
        rsi = latest['rsi']
        stoch_k = latest['stoch_k']
        stoch_d = latest['stoch_d']
        adx = latest['adx']
        ema_cross = latest['ema_fast'] > latest['ema_slow']
        volume_spike = latest['volume'] > latest['vol_ma'] * VOLUME_SPIKE_THRESHOLD
        pattern = latest['pattern']
        close = latest['close']
        stop_loss = df_15m['low'].iloc[-5]
        take_profit = close + (close - stop_loss) * 1.5
        tv_symbol = symbol.replace("/", "")
        now = time.time()

        ema_fast_1h = df_1h['ema_fast'].iloc[-1]
        ema_slow_1h = df_1h['ema_slow'].iloc[-1]
        higher_tf_uptrend = ema_fast_1h > ema_slow_1h

        if symbol not in last_alert_time:
            last_alert_time[symbol] = 0

        msg = ""

        # === LONG SIGNAL ===
        if rsi > 50 and stoch_k > stoch_d and ema_cross and volume_spike and adx < 25 and higher_tf_uptrend:
            msg += f"🔼iproved LONG Setup\nRSI: {rsi:.2f}, ADX: {adx:.2f}, Pattern: {pattern}"
            msg += f"\nEMA Cross: ✅\n🎯 TP: {take_profit:.2f}, 🛑 SL: {stop_loss:.2f}"
            if now - last_alert_time[symbol] > ALERT_COOLDOWN:
                send_alert(symbol, msg, tv_symbol)
                last_alert_time[symbol] = now

        # === SHORT SIGNAL ===
        elif rsi < 50 and stoch_k < stoch_d and not ema_cross and volume_spike and adx < 25 and not higher_tf_uptrend:
            stop_loss = df_15m['high'].iloc[-5]
            take_profit = close - (stop_loss - close) * 1.5
            msg += f"🔽iproved SHORT Setup\nRSI: {rsi:.2f}, ADX: {adx:.2f}, Pattern: {pattern}"
            msg += f"\nEMA Cross: ❌\n🎯 TP: {take_profit:.2f}, 🛑 SL: {stop_loss:.2f}"
            if now - last_alert_time[symbol] > ALERT_COOLDOWN:
                send_alert(symbol, msg, tv_symbol)
                last_alert_time[symbol] = now

        # === CANDLE PATTERN ONLY ALERT ===
        elif pattern in ['Bullish Engulfing', 'Bearish Engulfing'] and now - last_alert_time[symbol] > ALERT_COOLDOWN:
            send_alert(symbol, f"📉iproved Candlestick Pattern Detected: {pattern}\nClose: {close}", tv_symbol)
            last_alert_time[symbol] = now

        # === BREAKOUT ALERT ===
        range_high = df_15m['high'].rolling(20).max().iloc[-1]
        range_low = df_15m['low'].rolling(20).min().iloc[-1]
        if close > range_high and now - last_alert_time[symbol] > ALERT_COOLDOWN:
            send_alert(symbol, f"🚀 BREAKOUT ABOVE Resistance\nPrice: {close:.2f}", tv_symbol)
            last_alert_time[symbol] = now
        elif close < range_low and now - last_alert_time[symbol] > ALERT_COOLDOWN:
            send_alert(symbol, f"⚠️ BREAKDOWN BELOW Support\nPrice: {close:.2f}", tv_symbol)
            last_alert_time[symbol] = now

        # === DEBUG LOG ===
        print(f"[{datetime.now().strftime('%H:%M:%S')}] {symbol} | RSI: {rsi:.2f}, EMA: {ema_cross}, Vol: {volume_spike}, TF Uptrend: {higher_tf_uptrend}, Pattern: {pattern}, Close: {close:.4f}")

    except Exception as e:
        print(f"[Signal error] {symbol}: {e}")

# === MAIN LOOP ===
send_alert("BOT", "✅ Advanced Crypto Signal Bot Started...", "BTCUSDT")

while True:
    for symbol in symbols:
        check_signals(symbol)
        time.sleep(1)
    time.sleep(60)
